<a href="https://colab.research.google.com/github/adrianchespoch/Image_Processing/blob/main/Image_analysis_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image processing with Python

- This is a tutorial on the basics of image analysis with Python

## 1. Importing Modules & Packages

- Import the necesary packages

## 2. Loading Image Data and vizualize

*   Load all images in the /data folder (not one by one, but efficiently using a loop or list comprehension).
*   Visualize the images using subplots, arranging them in 5 rows and 2 columns (left column for noisy images and right column for the corresponding ground truth (GT) images).
*   Ensure the color bar range is the same for both columns, i.e., use the same color scale for all images.

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, filters
from skimage.metrics import mean_squared_error, structural_similarity
import cv2
from ipywidgets import interact, FloatSlider, IntSlider

# Función para leer imagen y convertir a float en [0,1]
def load_and_normalize(path):
    img = io.imread(path)
    # Convertir a float en [0,1] (asumimos que las imágenes son 8-bit)
    img_norm = img.astype(np.float32) / 255.0
    return img_norm

# Función para aplicar dos filtros pasa bajo:
# - Gaussiano (parámetro sigma)
# - Filtro de media (parámetro kernel_size)
def apply_low_pass_filters(noisy_img, sigma, kernel_size):
    # Filtro Gaussiano
    gauss_filtered = filters.gaussian(noisy_img, sigma=sigma)
    # Filtro de media: usamos cv2.blur, pero primero escalamos a 0-255 y luego volvemos a 0-1
    mean_filtered = cv2.blur((noisy_img*255).astype(np.uint8), (kernel_size, kernel_size))
    mean_filtered = mean_filtered.astype(np.float32) / 255.0
    return gauss_filtered, mean_filtered

# Función para calcular RMSE
def compute_rmse(gt, filtered):
    mse_val = mean_squared_error(gt, filtered)
    rmse = np.sqrt(mse_val)
    return rmse

# Función para calcular SSIM (asumimos que las imágenes están en [0,1])
def compute_ssim(gt, filtered):
    ssim_val, _ = structural_similarity(gt, filtered, full=True)
    return ssim_val

# Función que integra el filtrado y la visualización para una imagen (con su GT)
def low_pass_filtering_visualization(noisy_path, gt_path, sigma, kernel_size):
    # Cargar imágenes y normalizarlas
    noisy = load_and_normalize(noisy_path)
    gt = load_and_normalize(gt_path)

    # Aplicar filtros pasa bajo
    gauss_filtered, mean_filtered = apply_low_pass_filters(noisy, sigma, kernel_size)

    # Calcular métricas
    rmse_gauss = compute_rmse(gt, gauss_filtered)
    ssim_gauss = compute_ssim(gt, gauss_filtered)
    rmse_mean = compute_rmse(gt, mean_filtered)
    ssim_mean = compute_ssim(gt, mean_filtered)

    # Calcular escala global para visualización (mismo vmin/vmax en todas las imágenes)
    vmin = min(np.min(noisy), np.min(gt))
    vmax = max(np.max(noisy), np.max(gt))

    # Visualización en subplots: 1 fila, 4 columnas (Noisy, GT, Gaussian, Mean)
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))

    im0 = axes[0].imshow(noisy, cmap="gray", vmin=vmin, vmax=vmax)
    axes[0].set_title("Noisy")
    axes[0].axis("off")
    fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

    im1 = axes[1].imshow(gt, cmap="gray", vmin=vmin, vmax=vmax)
    axes[1].set_title("GT")
    axes[1].axis("off")
    fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

    im2 = axes[2].imshow(gauss_filtered, cmap="gray", vmin=vmin, vmax=vmax)
    axes[2].set_title(f"Gaussian\nRMSE: {rmse_gauss:.3f}\nSSIM: {ssim_gauss:.3f}")
    axes[2].axis("off")
    fig.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)

    im3 = axes[3].imshow(mean_filtered, cmap="gray", vmin=vmin, vmax=vmax)
    axes[3].set_title(f"Mean\nRMSE: {rmse_mean:.3f}\nSSIM: {ssim_mean:.3f}")
    axes[3].axis("off")
    fig.colorbar(im3, ax=axes[3], fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()

# Función interactiva para Low Pass Filtering para img4 y img5
def interactive_low_pass():
    # Especificar rutas para img4 y img5 y sus GT
    options = {
        "img4": {"noisy": "data/img4.tif", "gt": "data/img4_GT.tif"},
        "img5": {"noisy": "data/img5.jpg", "gt": "data/img5_GT.jpg"}
    }

    def low_pass(noisy_key, sigma, kernel_size):
        paths = options[noisy_key]
        low_pass_filtering_visualization(paths["noisy"], paths["gt"], sigma, kernel_size)

    interact(low_pass,
             noisy_key = {"img4": "img4", "img5": "img5"},
             sigma = FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description="Sigma"),
             kernel_size = IntSlider(value=5, min=3, max=15, step=2, description="Kernel Size"))

interactive_low_pass()


## 3. Spatial filtering

### Low pass Filtering

*   Select  two low-pass filters of your choice.
*   Use widgets for the interactive selection of the parameters and select the best values for img4 and img5 (independently). In other words, find the parameters that blur out the noise while keeping the structures of interest.
*  Calulate the RMSE and SSIM (Structural Similarity Index Measure) between the noisy image and the reulst of teh filtering with respect to the GT.
*   Visualize the best results using subplots. For each image, display:
    - The noisy image.
    - The corresponding ground truth (GT).
    - The results of the two filtering methods.
*   Ensure the color bar range is the same for both columns, i.e., use the same color scale for all images.
*   Print the values of RMSE and SSIM on teh corresponding images.   

### High pass Filtering

*   Select two high-pass filters of your choice.
*   Use widgets for the interactive selection of the parameters and choose the best values for img1 and img3 (independently). In other words, find the parameters that remove low-frequency noise while preserving the high-frequency structures of interest.
*   Visualize the results using subplots. For each image, display:
    - The noisy image.
    - The results of the two filtering methods.
*   Repeat the procesing for img1_GT and img3_GT. For img1_GT, first invert the image, i.e using the operaton img = 255 - img.
*   Calulate the Peak-Signal-To-Noise-Ratio (PSNR) for all images and print them over them in the plot

## 4. Border detection
*   Apply Sobel, Prewitt y Canny methods for edge detection to the images
*   Use widgets for the interactive selection of the parameters and choose the best values for img1 and img3 (independently) , i.e. for the Canny method
*   Visualize the results using subplots. For each image, display:
    - The noisy image.
    - The corresponding ground truth (GT).
    - The results of the two filtering methods.
*   Repeat the processing in the images after noise removal (i.e. after applying a low-pass filter) and compare the results.
*   Calulate the Peak-Signal-To-Noise-Ratio (PSNR) for all images and print them over them in the plot



## 4. Filtering in the frequency domain

### Low pass Filtering

*   Use the FFT to create a low-pass for images img4 and img5.
*   Use widgets for the interactive selection of the parameters and select the best values for img4 and img5 (independently).
*  Calulate the RMSE and SSIM (Structural Similarity Index Measure) between the noisy image and the results of the filtering with respect to the GT.
*   Visualize the best results using subplots. For each image, display:
    - The noisy image.
    - The corresponding ground truth (GT).
    - The results of the filtering.
*   Ensure the color bar range is the same for both columns, i.e., use the same color scale for all images.
*   Print the values of RMSE and SSIM on the corresponding images.     

### High pass Filtering

*   Use the FFT to create a high-pass for images img1 and img3.
*   Use widgets for the interactive selection of the parameters and choose the best values for img1 and img3 (independently).
*   Visualize the results using subplots. For each image, display:
    - The noisy image.
    - The results of the filtering .
*   Repeat the procesing for img1_GT and img3_GT. For img1_GT, first invert the image, i.e using the operaton img = 255 - img.
*   Calulate the Peak-Signal-To-Noise-Ratio (PSNR) for all images and print them over them in the plot

## 5. Results comparison



1.   Plot the results of all low-pass filters in a single figure for img4 and img5, as well as the original image and the GT
*   Print on each image the values of PSNR, RMSE and SSIM accordingly

2.   Plot the results of all high-pass filters in a single figure for img1 and img3, as well as the original image
*   Print on each image the values of PSNR accordingly

## 6. Evaluation in a complete dataset



*   Download the complete dataset from https://drive.google.com/file/d/1SxBDH2PCs8J0ifWpohOe5RSjS6R5-Ysx/view?usp=sharing
*   Apply the 3 low-pass filters used here with the 'best' parameters foeund before to all images in teh folder 'images'
*   Calculate the RMSE and SSIM relative to the corresponding GT images
*   Plot the results as box plots or violin plots for the noisy images and resulst of the 3 filtering methods.
*   Calulate the PSNR for all images, i.e. GT, noisy, and filtered
*   Plot the results as box plots or violin plots




## Example of a widget implementation for Interactive parameter selection

In [ ]:
# NumPy :  manipulation of numerical arrays:
import numpy as np

# The plotting module matplotlib.pyplot as plt
import matplotlib.pyplot as plt

# The image processing package scipy.ndimage as ndi
import scipy.ndimage as ndi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

filename = r'cells_2D.tif'
dirpath = r'/content/drive/MyDrive/IntroBioImageAnalysis-main/'
from os.path import join
filepath = join(dirpath, filename)
from skimage.io import imread
img = imread(filepath)


Mounted at /content/drive


In [ ]:
# Function to plot pairs of images

def pltPair(img1, img2, title1, title2, cmap1, cmap2, lim):
    """This function displays a pair of input image with interpolation='none',in the range of lims. """

    plt.figure(figsize=(16,16))
    plt.subplot(1,2,1)
    plt.imshow(img1, interpolation='none', cmap=cmap1, vmin=lim[0], vmax=lim[1])
    plt.title(title1)
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.subplot(1,2,2)
    plt.imshow(img2, interpolation='none', cmap=cmap2, vmin=lim[2], vmax=lim[3])
    plt.title(title2)
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.show()

In [ ]:
# Try out different thresholds to find the best one using 'widgets' for interactive selection

# Prepare widget
from ipywidgets import interact
@interact(sigma=(1,40,1))
def select_threshold(sigma=1):

    # Smoothing
    img_smooth = ndi.gaussian_filter(img, sigma)

    # Visualization
    pltPair(img, img_smooth, 'Raw Image', 'Smoothed Image with sigma =' + str(sigma), 'magma', 'magma', [0,255,0,255])



interactive(children=(IntSlider(value=1, description='sigma', max=40, min=1), Output()), _dom_classes=('widget…

----

*For some users, it is necessary to specifically activate the widgets plugin for Jupyter notebook. To do so, save and exit Jupyter notebook, then go to a terminal and write `jupyter nbextension enable --py --sys-prefix widgetsnbextension`.